## This script is used to compare the results from all 3 software.
## Written by Yusreen Shah
## Date: May 10th 2023

In [60]:
# import the libraries
import numpy as np
import pandas as pd
import re 
import Levenshtein
from Bio import SeqIO
from collections import defaultdict


## This section saves the list of all the queries in a dataframe

In [61]:
#Read the query and sequences from the .fasta file
fasta_file = "111_Graph/combined_1.fasta"  

# Create empty lists to store the data
queries = []
sequences = []

# Read through the FASTA file and extract query and sequence information
for record in SeqIO.parse(fasta_file, "fasta"):
    queries.append(record.id)
    sequences.append(str(record.seq))

# Create DataFrame from the lists
df = pd.DataFrame({'Query': queries, 'Sequence': sequences})

# Display the DataFrame
print(df)


                                                 Query   
0    gb|FJ503047|+|0-519|ARO:3002592|AAC(6')-Ib-Han...  \
1    gb|EU085533|+|0-519|ARO:3002591|AAC(6')-Ib-Suzhou   
2        gb|DQ303918|+|0-600|ARO:3002547|AAC(6')-Ib-cr   
3        gb|JQ808129|+|633-1188|ARO:3002546|AAC(6')-Ib   
4         gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10   
..                                                 ...   
373       gb|U00096|+|2367070-2368039|ARO:3003578|pmrF   
374           gb|JF969163|+|1053-1893|ARO:3000410|sul1   
375       gb|AY055428.1|-|20268-21084|ARO:3000412|sul2   
376   gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM   
377        gb|U00096|-|2098446-2099613|ARO:3003577|ugd   

                                              Sequence  
0    ATGACTGAGCATGACCTTGTGATGCTCTATGAGTGGCTAAATCGAT...  
1    ATGACTGAGCATGACCTTGCGATGCTCTATGAGTGGCTAAATCGAT...  
2    ATGAGCAACGCAAAAACAAAGTTAGGCATCACAAAGTACAGCATCG...  
3    GTGACCAACAGCAACGATTCCGTAACACTGCGCCTCATGACTGAGC...  
4    ATGTTACGCAGCA

In [62]:
# Remove Seq(' and ') from the sequences
df['Sequence'] = df['Sequence'].str.replace('Seq(''', '')
df['Sequence'] = df['Sequence'].str.replace(')', '')
df['Sequence'] = df['Sequence'].str.strip(" \' ")
df['Sequence']=df['Sequence'].str.rstrip()

In [63]:
df=df.sort_values("Query")
df=df.reset_index(drop=True)
df

,Query,Sequence
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...


In [64]:
df.to_csv("testSequences.tsv",sep="\t")

## This section is used to compare the results from Bandage and the actual list of queries.

In [6]:
# Create a dataframe for Bandage Combined1
Bandage_Combined1= pd.read_csv('111_GraphResults/Bandageoutputcombined1.tsv', sep='\t')
Bandage_Combined1

,Query,Path,Length,Query covered by path,Query covered by hits,Mean hit identity,Total hit mismatches,Total hit gap opens,Relative length,Length discrepancy,E-value product,Sequence
0,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,(3687) 118828- (4123),437,100%,100%,96.58%,14,1,99.7717%,-1,0.0,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
1,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,(3687) 118828- (4123),437,100%,100%,96.12%,16,1,99.7717%,-1,0.0,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
2,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,(378) 28874510+,935,96.6049%,96.6049%,99.47%,1,2,99.574%,-4,0.0,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...
3,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,(1326) 2007548+ (2129),804,100%,100%,99.88%,1,0,100%,0,0.0,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
4,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,(2129) 2007548+ (2965),837,100%,100%,99.76%,2,0,100%,0,0.0,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
...,...,...,...,...,...,...,...,...,...,...,...,...
57,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,(565) 136860+ (2484),1920,100%,100%,98.8%,23,0,100%,0,0.0,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
58,gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,(565) 136860+ (2477),1913,99.6354%,99.6354%,96.45%,66,2,100%,0,0.0,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
59,gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB,(46387) 2066834- (46980),594,100%,100%,98.82%,7,0,100%,0,0.0,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...
60,gb|U00096|-|2098446-2099613|ARO:3003577|ugd,"(523) 827680+, 120721+ (732)",1167,100%,100%,97.8718%,26,0,100%,0,0.0,ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...


In [7]:
# Perform a left merge on 'Query' column to compare sequences
merged_df_test = pd.merge(df,Bandage_Combined1, on='Query', how='right')

merged_df_test

,Query,Sequence_x,Path,Length,Query covered by path,Query covered by hits,Mean hit identity,Total hit mismatches,Total hit gap opens,Relative length,Length discrepancy,E-value product,Sequence_y
0,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...,(3687) 118828- (4123),437,100%,100%,96.58%,14,1,99.7717%,-1,0.0,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
1,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,ATGGACATCAGGCAAATGAACAAAACCCATCTGGAGCACTGGCGCG...,(3687) 118828- (4123),437,100%,100%,96.12%,16,1,99.7717%,-1,0.0,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
2,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,(378) 28874510+,935,96.6049%,96.6049%,99.47%,1,2,99.574%,-4,0.0,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...
3,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...,(1326) 2007548+ (2129),804,100%,100%,99.88%,1,0,100%,0,0.0,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
4,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,(2129) 2007548+ (2965),837,100%,100%,99.76%,2,0,100%,0,0.0,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,(565) 136860+ (2484),1920,100%,100%,98.8%,23,0,100%,0,0.0,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
58,gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,(565) 136860+ (2477),1913,99.6354%,99.6354%,96.45%,66,2,100%,0,0.0,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
59,gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB,ATGATCATTTGGATAAACGGGGCATTCGGTTCGGGAAAAACACAAA...,(46387) 2066834- (46980),594,100%,100%,98.82%,7,0,100%,0,0.0,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...
60,gb|U00096|-|2098446-2099613|ARO:3003577|ugd,ATGAAAATCACCATTTCCGGTACTGGCTATGTAGGCTTGTCAAACG...,"(523) 827680+, 120721+ (732)",1167,100%,100%,97.8718%,26,0,100%,0,0.0,ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...


In [9]:
# Calculate the difference between 'Sequence_x' and 'Sequence_y'
merged_df_test['Distance'] = Levenshtein.distance(merged_df_test['Sequence_x'],merged_df_test['Sequence_y'])
# Create a new dataframe with unique values in the 'Query' column
df_unique = df.drop_duplicates(subset='Query', keep='first')


#merged_df_test.to_csv("CAMIH1_Results/TestDistance.tsv",sep="\t")
# Print the new dataframe
df_unique
df_unique.to_csv("CAMIM2_Results/TestDistance.tsv",sep="\t")

In [10]:
df_unique

,Query,Sequence
0,gb|AB765395|+|0-1143|ARO:3002993|AQU-1,ATGAAGCAAACCTCACCCTTGTCGTCGCTGGCGCTGAGCGCCCTGC...
1,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
2,gb|AE016877.1|+|1972251-1972668|ARO:3000172|FosB,TTGTTAAGGGGAATCAATCATATTTGTTTTTCGGTATCTAATTTAG...
3,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
4,gb|AF118110.1|-|71-1037|ARO:3003002|CfxA2,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...
5,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,ATGGACATCAGGCAAATGAACAAAACCCATCTGGAGCACTGGCGCG...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF472622|+|52-1018|ARO:3003003|CfxA3,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...
8,gb|AF550679.1|-|80976-81768|ARO:3002601|aadA,ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGG...
9,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


In [11]:
#len(df.index)
Bandage_Combined1.dtypes

Query                     object
Path                      object
Length                     int64
Query covered by path     object
Query covered by hits     object
Mean hit identity         object
Total hit mismatches       int64
Total hit gap opens        int64
Relative length           object
Length discrepancy         int64
E-value product          float64
Sequence                  object
dtype: object

In [12]:
Bandage_Combined_Query_Path =Bandage_Combined1[['Path','Query','Sequence']]

## Extract the start and end position from the path in Bandage

In [13]:
# Extract the start position
pattern_path_start = r'\((.*?)\)' 

In [14]:
#Extract the node position
pattern_path_node = r'\b(\d+)\s*[+-]'

In [15]:
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Path'].apply(lambda x: re.findall(pattern_path_node, x))

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/75317959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Path'].apply(lambda x: re.findall(pattern_path_node, x))


In [16]:
# Remove the braces and convert the values to integers
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].apply(lambda x: [int(value) for value in x])

Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].astype(str)
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].str.replace('[', '').str.replace(']', '')


/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/1862839196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].apply(lambda x: [int(value) for value in x])
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/1862839196.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].astype(str)
/var/folders/

In [17]:

Bandage_Combined_Query_Path.to_csv("test4.tsv",sep="\t")

In [18]:
Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)
Bandage_Combined_Query_Path

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/3300280694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)


,Path,Query,Sequence,Extracted_Path,Start
0,(3687) 118828- (4123),gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...,118828,3687
1,(3687) 118828- (4123),gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...,118828,3687
2,(378) 28874510+,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...,28874510,378
3,(1326) 2007548+ (2129),gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...,2007548,1326
4,(2129) 2007548+ (2965),gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,2007548,2129
...,...,...,...,...,...
57,(565) 136860+ (2484),gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,136860,565
58,(565) 136860+ (2477),gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,136860,565
59,(46387) 2066834- (46980),gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...,2066834,46387
60,"(523) 827680+, 120721+ (732)",gb|U00096|-|2098446-2099613|ARO:3003577|ugd,ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...,"827680, 120721",523


In [19]:
# Extract the end position
Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]


/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/363825808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]


In [20]:
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('+')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('-')
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].astype(int)
Bandage_Combined_Query_Path

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/4133168393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()')
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/4133168393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('+')
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/4133168393.py:3: Settin

,Path,Query,Sequence,Extracted_Path,Start,End
0,(3687) 118828- (4123),gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...,118828,3687,4123
1,(3687) 118828- (4123),gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...,118828,3687,4123
2,(378) 28874510+,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...,28874510,378,28874510
3,(1326) 2007548+ (2129),gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...,2007548,1326,2129
4,(2129) 2007548+ (2965),gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,2007548,2129,2965
...,...,...,...,...,...,...
57,(565) 136860+ (2484),gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,136860,565,2484
58,(565) 136860+ (2477),gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,136860,565,2477
59,(46387) 2066834- (46980),gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...,2066834,46387,46980
60,"(523) 827680+, 120721+ (732)",gb|U00096|-|2098446-2099613|ARO:3003577|ugd,ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...,"827680, 120721",523,732


In [21]:
# Reorder the columns in Bandage_Combined_Query_Path
Bandage_Combined_Query_Path=Bandage_Combined_Query_Path[['Query','Path','Start','Extracted_Path','End','Sequence']]

# Convert integer columns to strings
Bandage_Combined_Query_Path['Path'] = Bandage_Combined_Query_Path['Path'].astype(str)
Bandage_Combined_Query_Path['End'] = Bandage_Combined_Query_Path['End'].astype(str)
Bandage_Combined_Query_Path['Start'] = Bandage_Combined_Query_Path['Start'].astype(str)
Bandage_Combined_Query_Path['Extracted_Path'] = Bandage_Combined_Query_Path['Extracted_Path'].astype(str)
Bandage_Combined_Query_Path['Sequence'] = Bandage_Combined_Query_Path['Sequence'].astype(str)

# Bandage_Combined1['Query'].value_counts() 
# Merge the rows that have the same query
# group the dataframe by the 'Name' column and aggregate the data for each group
merge_Bandage_df = Bandage_Combined_Query_Path.groupby('Query').agg({'Path': ' , '.join, 'Start': ', '.join, 'End': ', '.join, 'Extracted_Path': ','.join, 'Sequence': ' '.join}).reset_index()
merge_Bandage_df


,Query,Path,Start,End,Extracted_Path,Sequence
0,gb|AB765395|+|0-1143|ARO:3002993|AQU-1,(24879) 2079546- (26021),24879,26021,2079546,ATGAAGCAAACCTCACCCTTGTCGTTGCTGGCGCTGAGCGCCCTGC...
1,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,(3687) 118828- (4123),3687,4123,118828,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
2,gb|AE016877.1|+|1972251-1972668|ARO:3000172|FosB,"(311) 15445+ (727) , (310) 2021892+ (722)","311, 310","727, 722","15445,2021892",TTGTTAAGGGGAATCAATCATATTTGTTTTTCGGTATCTAATTTAG...
3,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,(2129) 2007548+ (2965),2129,2965,2007548,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
4,gb|AF118110.1|-|71-1037|ARO:3003002|CfxA2,(112) 821183+ (1077),112,1077,821183,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...
5,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,(3687) 118828- (4123),3687,4123,118828,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,(1326) 2007548+ (2129),1326,2129,2007548,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF472622|+|52-1018|ARO:3003003|CfxA3,(112) 821183+ (1077),112,1077,821183,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...
8,gb|AF550679.1|-|80976-81768|ARO:3002601|aadA,(554) 28874510+,554,28874510,28874510,ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGG...
9,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,(565) 136860+ (2484),565,2484,136860,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


In [22]:
# Add Braces to the values that now have ','
def add_braces(value):
    if ',' in value:
        return '{' + value + '}'
    else:
        return value
#merge_Bandage_df['Path'] = merge_Bandage_df['Path'].apply(add_braces)
#merge_Bandage_df['Start'] = merge_Bandage_df['Start'].apply(add_braces)
#merge_Bandage_df['End'] = merge_Bandage_df['End'].apply(add_braces)
#merge_Bandage_df['Sequence'] = merge_Bandage_df['Sequence'].apply(add_braces)

In [23]:
# Store the dataframe containing the Bandage results to a .tsv file
merge_Bandage_df.to_csv("SequenceBandage.tsv", sep="\t")

In [24]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsBandage=pd.merge(df,merge_Bandage_df, on='Query',how="outer")

## Compile the Queries and the results from Bandage in one dataframe

In [25]:
df_QueryAndResultsBandage

,Query,Sequence_x,Path,Start,End,Extracted_Path,Sequence_y
0,gb|AB765395|+|0-1143|ARO:3002993|AQU-1,ATGAAGCAAACCTCACCCTTGTCGTCGCTGGCGCTGAGCGCCCTGC...,(24879) 2079546- (26021),24879,26021,2079546,ATGAAGCAAACCTCACCCTTGTCGTTGCTGGCGCTGAGCGCCCTGC...
1,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...,(3687) 118828- (4123),3687,4123,118828,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
2,gb|AE016877.1|+|1972251-1972668|ARO:3000172|FosB,TTGTTAAGGGGAATCAATCATATTTGTTTTTCGGTATCTAATTTAG...,"(311) 15445+ (727) , (310) 2021892+ (722)","311, 310","727, 722","15445,2021892",TTGTTAAGGGGAATCAATCATATTTGTTTTTCGGTATCTAATTTAG...
3,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,(2129) 2007548+ (2965),2129,2965,2007548,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
4,gb|AF118110.1|-|71-1037|ARO:3003002|CfxA2,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...,(112) 821183+ (1077),112,1077,821183,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...
5,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,ATGGACATCAGGCAAATGAACAAAACCCATCTGGAGCACTGGCGCG...,(3687) 118828- (4123),3687,4123,118828,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...,(1326) 2007548+ (2129),1326,2129,2007548,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF472622|+|52-1018|ARO:3003003|CfxA3,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...,(112) 821183+ (1077),112,1077,821183,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...
8,gb|AF550679.1|-|80976-81768|ARO:3002601|aadA,ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGG...,(554) 28874510+,554,28874510,28874510,ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGG...
9,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,(565) 136860+ (2484),565,2484,136860,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


In [26]:
#Print the columns from the dataframe
df_QueryAndResultsBandage.columns

Index(['Query', 'Sequence_x', 'Path', 'Start', 'End', 'Extracted_Path',
       'Sequence_y'],
      dtype='object')

In [27]:
#Rename the columns from the merging of Bandage and Query list
df_QueryAndResultsBandage.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Sequence_y':'Sequence_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Length':'Length_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Path':'Path_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Start':'Start_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'End':'End_Bandage'}, inplace = True)

## Save the Queries and Results from Bandage in one .tsv file

In [28]:
df_QueryAndResultsBandage.to_csv("CompiledQueriesBandage.tsv",sep="\t")
df_QueryAndResultsBandage

,Query,Sequence,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage
0,gb|AB765395|+|0-1143|ARO:3002993|AQU-1,ATGAAGCAAACCTCACCCTTGTCGTCGCTGGCGCTGAGCGCCCTGC...,(24879) 2079546- (26021),24879,26021,2079546,ATGAAGCAAACCTCACCCTTGTCGTTGCTGGCGCTGAGCGCCCTGC...
1,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...,(3687) 118828- (4123),3687,4123,118828,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
2,gb|AE016877.1|+|1972251-1972668|ARO:3000172|FosB,TTGTTAAGGGGAATCAATCATATTTGTTTTTCGGTATCTAATTTAG...,"(311) 15445+ (727) , (310) 2021892+ (722)","311, 310","727, 722","15445,2021892",TTGTTAAGGGGAATCAATCATATTTGTTTTTCGGTATCTAATTTAG...
3,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,(2129) 2007548+ (2965),2129,2965,2007548,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
4,gb|AF118110.1|-|71-1037|ARO:3003002|CfxA2,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...,(112) 821183+ (1077),112,1077,821183,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...
5,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,ATGGACATCAGGCAAATGAACAAAACCCATCTGGAGCACTGGCGCG...,(3687) 118828- (4123),3687,4123,118828,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...,(1326) 2007548+ (2129),1326,2129,2007548,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF472622|+|52-1018|ARO:3003003|CfxA3,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...,(112) 821183+ (1077),112,1077,821183,ATGGAAAAAAACAGAAAAAAACAAATCGTAGTTTTGAGTATAGCTT...
8,gb|AF550679.1|-|80976-81768|ARO:3002601|aadA,ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGG...,(554) 28874510+,554,28874510,28874510,ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGG...
9,gb|AJ222769.3|+|3686-5606|ARO:3000194|tetW,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,(565) 136860+ (2484),565,2484,136860,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...


## This section is used to compare the results from SPAligner and the actual list of queries.

In [29]:
#Create a dataframe for SPAligner Combined1
SPAligner_Combined1=pd.read_csv('CAMIM2_Results/outputCAMIM2SPAligner.tsv', sep='\t',names=["Query","Start position of alignment on sequence","End position of alignment on sequence","Start position of alignment on the first edge of the path",
"End position of alignment on the last edge of the path","Path of alignment","Length of the alignment on each edge of the path","Sequence Length",
"Sequence"])
SPAligner_Combined1

,Query,Start position of alignment on sequence,End position of alignment on sequence,Start position of alignment on the first edge of the path,End position of alignment on the last edge of the path,Path of alignment,Length of the alignment on each edge of the path,Sequence Length,Sequence
0,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,0,383,3686,4068,438,118828-,382,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
1,gb|L12710|+|0-549|ARO:3002556|AAC(6')-Ii [Ente...,220,475,0,255,549,28611963-,255,ATCCATTAGTTGTAGAAAGCTCCCGACGAAAGAACCAAATAGGTAC...
2,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy...,0,383,3686,4068,438,118828-,382,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
3,gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,0,1858,564,2422,1920,136860+,1858,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
4,gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB...,0,539,46386,46925,594,2066834-,539,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...
5,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-II...,0,884,377,1257,972,28874510+,880,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')...,0,749,1325,2074,804,2007548+,749,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id ...,0,782,2128,2910,837,2007548+,782,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
8,gb|U00096|-|2098446-2099613|ARO:3003577|ugd [E...,0,1112,522,677,1167,"827680+,120721+","435,677",ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...
9,gb|AB765395|+|0-1143|ARO:3002993|AQU-1 [Aeromo...,0,1088,24878,25966,1143,2079546-,1088,ATGAAGCAAACCTCACCCTTGTCGTTGCTGGCGCTGAGCGCCCTGC...


In [30]:
#get the columns from SPAligner
SPAligner_Combined1.columns

Index(['Query', 'Start position of alignment on sequence',
       'End position of alignment on sequence',
       'Start position of alignment on the first edge of the path',
       'End position of alignment on the last edge of the path',
       'Path of alignment', 'Length of the alignment on each edge of the path',
       'Sequence Length', 'Sequence'],
      dtype='object')

In [31]:
#format the Query column from  SPAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


SPAligner_Combined1['Query'] = SPAligner_Combined1['Query'].map(format_query)


In [32]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Sequence']]


In [33]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Start position of alignment on the first edge of the path', 'End position of alignment on the last edge of the path', 'Sequence Length',
'Path of alignment','Sequence']]
SPAligner_Combined_Query_Path

,Query,Start position of alignment on the first edge of the path,End position of alignment on the last edge of the path,Sequence Length,Path of alignment,Sequence
0,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,3686,4068,382,438,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
1,gb|L12710|+|0-549|ARO:3002556|AAC(6')-Ii,0,255,255,549,ATCCATTAGTTGTAGAAAGCTCCCGACGAAAGAACCAAATAGGTAC...
2,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,3686,4068,382,438,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
3,gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,564,2422,1858,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
4,gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB,46386,46925,539,594,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...
5,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,377,1257,880,972,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,1325,2074,749,804,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,2128,2910,782,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
8,gb|U00096|-|2098446-2099613|ARO:3003577|ugd,522,677,"435,677",1167,ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...
9,gb|AB765395|+|0-1143|ARO:3002993|AQU-1,24878,25966,1088,1143,ATGAAGCAAACCTCACCCTTGTCGTTGCTGGCGCTGAGCGCCCTGC...


In [34]:
#Rename the columns in SPAligner
SPAligner_Combined_Query_Path.rename(columns = {'Start position of alignment on the first edge of the path':'Start_SPAligner'}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'End position of alignment on the last edge of the path':'End_SPAligner'}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Sequence Length':"Length_SPAligner"}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Path of alignment':"Path_SPAligner"}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Sequence':"Sequence_SPAligner"}, inplace = True)

SPAligner_Combined_Query_Path

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/2596825179.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAligner_Combined_Query_Path.rename(columns = {'Start position of alignment on the first edge of the path':'Start_SPAligner'}, inplace = True)
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/2596825179.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAligner_Combined_Query_Path.rename(columns = {'End position of alignment on the last edge of the path':'End_SPAligner'}, inplace = True)
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_4180/2596825179.py:4: SettingW

,Query,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,3686,4068,382,438,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
1,gb|L12710|+|0-549|ARO:3002556|AAC(6')-Ii,0,255,255,549,ATCCATTAGTTGTAGAAAGCTCCCGACGAAAGAACCAAATAGGTAC...
2,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,3686,4068,382,438,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
3,gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,564,2422,1858,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
4,gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB,46386,46925,539,594,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...
5,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,377,1257,880,972,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,1325,2074,749,804,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,2128,2910,782,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
8,gb|U00096|-|2098446-2099613|ARO:3003577|ugd,522,677,"435,677",1167,ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...
9,gb|AB765395|+|0-1143|ARO:3002993|AQU-1,24878,25966,1088,1143,ATGAAGCAAACCTCACCCTTGTCGTTGCTGGCGCTGAGCGCCCTGC...


In [35]:
SPAligner_Combined_Query_Path

,Query,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,3686,4068,382,438,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
1,gb|L12710|+|0-549|ARO:3002556|AAC(6')-Ii,0,255,255,549,ATCCATTAGTTGTAGAAAGCTCCCGACGAAAGAACCAAATAGGTAC...
2,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,3686,4068,382,438,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
3,gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,564,2422,1858,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
4,gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB,46386,46925,539,594,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...
5,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,377,1257,880,972,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,1325,2074,749,804,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,2128,2910,782,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
8,gb|U00096|-|2098446-2099613|ARO:3003577|ugd,522,677,"435,677",1167,ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...
9,gb|AB765395|+|0-1143|ARO:3002993|AQU-1,24878,25966,1088,1143,ATGAAGCAAACCTCACCCTTGTCGTTGCTGGCGCTGAGCGCCCTGC...


## Compile the Queries and the results from SPAligner in one dataframe

In [36]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsSPAligner=pd.merge(df,SPAligner_Combined_Query_Path, on='Query',how='right')

df_QueryAndResultsSPAligner

,Query,Sequence,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|AE006468.2|+|1707351-1707789|ARO:3002571|AA...,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...,3686,4068,382,438,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
1,gb|L12710|+|0-549|ARO:3002556|AAC(6')-Ii,ATGATAATCAGTGAATTTGACCGTAATAATCCAGTATTGAAAGATC...,0,255,255,549,ATCCATTAGTTGTAGAAAGCTCCCGACGAAAGAACCAAATAGGTAC...
2,gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy,ATGGACATCAGGCAAATGAACAAAACCCATCTGGAGCACTGGCGCG...,3686,4068,382,438,ATGGACATCAGGCAAATGAACAGAACCCATCTGGATCACTGGCGCG...
3,gb|KU736867.1|+|19653-21573|ARO:3004442|tet(W/...,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...,564,2422,1858,1920,ATGAAAATAATCAATATTGGAATTCTTGCCCATGTAGACGCTGGAA...
4,gb|AL009126.3|-|339155-339749|ARO:3003059|tmrB,ATGATCATTTGGATAAACGGGGCATTCGGTTCGGGAAAAACACAAA...,46386,46925,539,594,ATGATCATTTGGATAAACGGGGCATTCGGTTCCGGAAAAACACAAA...
5,gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTCTTGTTATGACAT...,377,1257,880,972,GTGGTAACGGCGCAGTGGCGGTTTTCATGGCTTGTTATGACTGTTT...
6,gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-Ib,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...,1325,2074,749,804,TTGAATCGAACTAATATTTTTTTTGGTGAATCGCATTCTGACTGGT...
7,gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...,2128,2910,782,837,ATGTTCATGCCGCCTGTTTTTCCTGCTCATTGGCACGTTTCGCAAC...
8,gb|U00096|-|2098446-2099613|ARO:3003577|ugd,ATGAAAATCACCATTTCCGGTACTGGCTATGTAGGCTTGTCAAACG...,522,677,"435,677",1167,ATGAAAATCACCATTTCCGGTACTGGCTATGTCGGCTTGTCAAACG...
9,gb|AB765395|+|0-1143|ARO:3002993|AQU-1,ATGAAGCAAACCTCACCCTTGTCGTCGCTGGCGCTGAGCGCCCTGC...,24878,25966,1088,1143,ATGAAGCAAACCTCACCCTTGTCGTTGCTGGCGCTGAGCGCCCTGC...


In [37]:
df_QueryAndResultsSPAligner.to_csv("TestCompiledQueriesSPAligner.tsv",sep="\t")

In [38]:
#Rename the columns from SPAligner and Results from the Query list
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_y':'Sequence_SPAligner'}, inplace = True)

In [39]:
df_QueryAndResultsSPAligner['Start_SPAligner']=df_QueryAndResultsSPAligner['Start_SPAligner']+1

df_QueryAndResultsSPAligner

TypeError: can only concatenate str (not "int") to str

In [40]:
df_QueryAndResultsSPAligner.to_csv("TestCompiledQueriesSPAligner.tsv",sep="\t")

## This section is used to compare the results from GraphAligner and the actual list of queries.

In [41]:
#Create a dataframe for GraphAligner Combined1
GraphAligner_Combined1=pd.read_csv('CAMIM2_Results/outputCAMIM2GraphAligner.tsv', sep='\t', names=["Query", "Query Length", "Query Start", 
                                        "Query End","Strand Relative Length","Path Matching","Path Length",
                                         "Start Position on Path","End Position on Path","Number of residues Matches",
                                         "Alignment Back Length","Mapping Quality","Column 1"])
GraphAligner_Combined1

Query   
gb|AE006468.2|+|1707351-1707789|ARO:3002571|AAC... 438  0    438      +  \
gb|L12710|+|0-549|ARO:3002556|AAC(6')-Ii [Enter... 549  220  530      +   
                                                        0    140      +   
gb|AF144880|+|3541-3979|ARO:3002569|AAC(6')-Iy ... 438  0    438      +   
gb|X02340.1|+|222-1194|ARO:3004089|ANT(3'')-IIa... 972  0    939      +   
gb|AF313472|+|15593-16397|ARO:3002639|APH(3'')-... 804  0    804      +   
gb|AF024602|+|3155-3992|ARO:3002660|APH(6)-Id [... 837  0    837      +   
gb|AB765395|+|0-1143|ARO:3002993|AQU-1 [Aeromon... 1143 0    1143     +   
gb|KF730243.1|+|0-1143|ARO:3004647|AQU-2 [Aerom... 1143 0    1143     +   
gb|KF730244.1|+|0-1149|ARO:3004648|AQU-3 [Aerom... 1149 0    1149     +   
gb|AL009126|+|916777-919348|ARO:3003324|Bacillu... 2571 0    2571     +   
gb|X06599|+|272-1193|ARO:3002877|BcI [Bacillus ... 921  0    921      +   
gb|KF526113|+|0-1146|ARO:3002112|CMY-100 [Citro... 1146 0    1146     +   
gb|KF526114|+|0-1146|ARO:3002113|CMY-101 [Esche... 1146 0    1146     +   
gb|JX440350|+|1026-2172|ARO:3002083|CMY-70 [Cit... 1146 0    1146     +   
gb|JX440349|+|1026-2172|ARO:3002087|CMY-74 [Cit... 1146 0    1146     +   
gb|KJ207203|+|5-1151|ARO:3002095|CMY-82 [Citrob... 1146 0    1146     +   
gb|JX440351|+|1026-2172|ARO:3002096|CMY-83 [Cit... 1146 0    1146     +   
gb|KF992025|+|1026-2172|ARO:3002106|CMY-93 [Cit... 1146 0    1146     +   
gb|GQ343019|+|132-1023|ARO:3002999|CblA-1 [mixe... 891  0    891      +   
gb|U38243|+|149-1115|ARO:3003001|CfxA [Bacteroi... 966  0    966      +   
gb|AF118110.1|-|71-1037|ARO:3003002|CfxA2 [Prev... 966  0    966      +   
gb|AF472622|+|52-1018|ARO:3003003|CfxA3 [Capnoc... 966  0    966      +   
gb|AY769933|+|0-966|ARO:3003005|CfxA4 [Bacteroi... 966  0    966      +   
gb|AY769934|+|27-993|ARO:3003096|CfxA5 [Parabac... 966  0    966      +   
gb|L42817|+|201-936|ARO:3000522|ErmG [Bacteroid... 735  321  642      +   
                                                        0    280      +   
gb|M36726.1|+|295-1150|ARO:3000596|ErmX [Plasmi... 855  0    855      +   
gb|U00096.3|-|4377810-4378944|ARO:3004290|Esche... 1134 0    1134     +   
gb|FN649414.1|+|2765050-2766355|ARO:3004611|Esc... 1305 0    1305     +   
gb|AP012030.1|-|395553-396711|ARO:3004612|Esche... 1158 309  1158     +   
gb|AE016877.1|+|1972251-1972668|ARO:3000172|Fos... 417  0    417      +   
gb|APRY01000059.1|+|64996-65827|ARO:3001740|OXA... 831  0    831      +   
gb|AJ276031.1|+|111-906|ARO:3004837|OXA-724 [Ae... 795  0    795      +   
gb|AF550679.1|-|80976-81768|ARO:3002601|aadA [E... 792  0    759      +   
gb|DQ393783|+|1799-2591|ARO:3002615|aadA15 [Pse... 792  0    748      +   
gb|AY171244|+|46-838|ARO:3002618|aadA21 [Salmon... 792  0    759      +   
gb|AM261837|+|73-865|ARO:3002619|aadA22 [Salmon... 792  0    759      +   
gb|AJ809407|+|118-898|ARO:3002620|aadA23 [Salmo... 780  8    747      +   
gb|DQ677333|+|0-780|ARO:3002621|aadA24 [Salmone... 780  8    747      +   
gb|AL009126|-|2735681-2736536|ARO:3002627|aadK ... 855  0    855      +   
gb|U00096.3|-|3203309-3204131|ARO:3002986|bacA ... 822  44   601      +   
gb|U60294|+|0-765|ARO:3003099|cphA2 [Aeromonas ... 765  0    765      +   
gb|AP009048|-|4338624-4340268|ARO:3003576|eptA ... 1644 0    1644     +   
gb|AJ548797|+|923-1688|ARO:3003094|imiH [Aeromo... 765  0    765      +   
gb|U00096.3|-|721055-721733|ARO:3003841|kdpE [E... 678  0    678      +   
gb|AJ579365.1|+|4149-5628|ARO:3003111|lsaB [Sta... 1479 0    1479     +   
gb|CP010314.1|+|275837-276758|ARO:3004541|mphK ... 921  0    921      +   
gb|U00096|+|2367070-2368039|ARO:3003578|pmrF [E... 969  0    969      +   
gb|AY055428.1|-|20268-21084|ARO:3000412|sul2 [V... 816  0    816      +   
gb|EU722333.1|+|1932-3852|ARO:3000196|tet32 [Es... 1920 949  1539     +   
                                                        453  967      +   
                                                        201  531      +   
                

In [42]:
#Format the Query column from  GraphAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


GraphAligner_Combined1['Query'] = GraphAligner_Combined1['Query'].map(format_query)

In [43]:
# Remove < and > from the Path
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('>', " ")
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('<', " ")

AttributeError: Can only use .str accessor with string values!

## This section drops the rows with coverages < 50 from GraphAligner_Combined1

In [ ]:
# Calculate the coverage for GraphAligner
GraphAligner_Combined1["Coverage"]=GraphAligner_Combined1["Alignment Back Length"]/GraphAligner_Combined1["Query Length"]*100

In [ ]:
GraphAligner_Combined1

In [ ]:
#Drop the coverages that are < 50 from GraphAligner 
GraphAligner_Combined1.drop(GraphAligner_Combined1[GraphAligner_Combined1['Coverage'] < 50].index, inplace = True)


In [ ]:
GraphAligner_Combined1['Start Position on Path']=GraphAligner_Combined1['Start Position on Path']+1
GraphAligner_Combined1['End Position on Path']=GraphAligner_Combined1['End Position on Path']
GraphAligner_Combined1

In [ ]:
#Concatenate the values from Path_Matching
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].apply(lambda x: ', '.join(x.split()))


In [ ]:
#Get the specific columns from GraphAligner
GraphAligner_Combined_Query_Path=GraphAligner_Combined1[['Query','Path Matching', 'Start Position on Path', 
'End Position on Path']]
# Convert the integer to string
GraphAligner_Combined_Query_Path['Query']=GraphAligner_Combined_Query_Path['Query'].astype(str)
GraphAligner_Combined_Query_Path['Path Matching']=GraphAligner_Combined_Query_Path['Path Matching'].astype(str)
GraphAligner_Combined_Query_Path['Start Position on Path']=GraphAligner_Combined_Query_Path['Start Position on Path'].astype(str)
GraphAligner_Combined_Query_Path['End Position on Path']=GraphAligner_Combined_Query_Path['End Position on Path'].astype(str)



In [ ]:
#Use merge operation so that we have the sequences for both softwares for successful queries
merged_df_Graph = GraphAligner_Combined_Query_Path.groupby('Query').agg({'Path Matching':' '.join, 'Start Position on Path': ', '.join,'End Position on Path': ', '.join}).reset_index()
merged_df_Graph

In [ ]:
merged_df_Graph.rename(columns = {'Path Matching':'Path_GraphAligner'}, inplace = True)
merged_df_Graph.rename(columns = {'Start Position on Path':'Start_GraphAligner'}, inplace = True)
merged_df_Graph.rename(columns = {'End Position on Path':'End_GraphAligner'}, inplace = True)

In [ ]:
merged_df_Graph

## Compile the Queries and the results from GraphAligner in one dataframe

In [ ]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsGraphAligner=pd.merge(df,merged_df_Graph, on='Query',how="left")
df_QueryAndResultsGraphAligner

## Compile all the results in one master Dataframe

In [ ]:
merged_df_1 = pd.merge(df_QueryAndResultsBandage, df_QueryAndResultsSPAligner, on=['Query', 'Sequence'], how='outer')
merged_df_master = pd.merge(merged_df_1, df_QueryAndResultsGraphAligner, on=['Query', 'Sequence'], how='outer')
#merged_df_master['Start_SPAligner']=merged_df_master['Start_SPAligner'].astype(float).astype(int)
merged_df_master

In [ ]:
#Drop some columns

merged_df_master.drop(['Sequence_y', 'Length_x','Length_y','Sequence'], axis=1)

#Rename Sequence_x
merged_df_master.rename(columns = {'Sequence_x':'Sequence_Query'}, inplace = True)



In [ ]:
def compare_paths(Start_Bandage, End_Bandage,Start_GraphAligner,End_GraphAligner):
    bandage_path_start = Start_Bandage.split(',')
    bandage_path_end = End_Bandage.split(',')
    #str(Start_GraphAligner) in str(bandage_path_start) and  str(End_GraphAligner) in  str(bandage_path_end):
    
    
    if set(str(bandage_path_start)) == set([str(Start_GraphAligner)]) and set(str(bandage_path_end) )== set([str(End_GraphAligner)]):
        return 'Similar'
    elif str(Start_GraphAligner) in str(bandage_path_start) and  str(End_GraphAligner) in  str(bandage_path_end):
        return 'Full'
    else:
        return 'Different'

In [ ]:
merged_df_master['BandageVSGraphAligner'] = merged_df_master.apply(lambda row: compare_paths(row['Start_Bandage'], row['End_Bandage'],row['Start_GraphAligner'],row['End_GraphAligner']), axis=1)

merged_df_master

In [ ]:
#Compare the Start, and end path from Bandage and SPAligner
merged_df_master['Start_SPAligner']=merged_df_master['Start_SPAligner'].astype(str)
merged_df_master['End_SPAligner']=merged_df_master['End_SPAligner'].astype(str)
merged_df_master['BandageVSSPAligner']=(merged_df_master['Start_Bandage'] == merged_df_master['Start_SPAligner']) & (merged_df_master['End_Bandage'] == merged_df_master['End_SPAligner'])



merged_df_master.dtypes

In [ ]:
#Compare the Start, and end path from GraphAligner and SPAligner
merged_df_master['SPAlignerVSGraphAligner']=(merged_df_master['Start_SPAligner'] == merged_df_master['Start_GraphAligner']) & (merged_df_master['End_SPAligner'] == merged_df_master['End_GraphAligner'])

In [ ]:
#Drop some columns

merged_df_master=merged_df_master.drop(['Sequence_y', 'Length_x','Length_y','Sequence'], axis=1)

## Save the Queries and  all the Results  in one .tsv file


In [ ]:
merged_df_master.to_csv("QueriesAndResultsMasterList.tsv",sep='\t')

In [ ]:
merged_df_master['Sequence_Bandage']=merged_df_master['Sequence_Bandage'].astype(str)

In [ ]:
def extract_closest_string(df):
    df['Extracted string'] = ''

    for index, row in df.iterrows():
        sequences = row['Sequence_Bandage']
        query = row['Sequence']
        closest_distance = float('inf')
        closest_string = ''

        if sequences:
            sequences = sequences.split(' ')
            for sequence in sequences:
                distance = Levenshtein.distance(sequence, query)
                if Levenshtein.distance(sequence, query) < closest_distance:
                    closest_distance = distance
                    closest_string = sequence

        df.at[index, 'Extracted string'] = closest_string


    return df

## This section saves the Nodes and Sequences from the gfa file

In [ ]:
import gfapy

# Open the GFA file
file_path = "graph1.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [ ]:
data_graph = []

In [ ]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

In [ ]:
#Count the number of duplicates
#len(df_graph['Name'])-len(df_graph['Name'].drop_duplicates())

In [ ]:
num_segments

In [ ]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [ ]:
df_graph

In [ ]:
df_graph['Name']=df_graph['Name'].astype(str)


## Get the sequences unique to GraphAligner
## Get the dataframe containing the path from GraphAligner


In [ ]:
df_GraphAligner_Paths= pd.read_csv('GraphAlignerNotEqual.tsv', sep='\t')
#GraphAlignerNotEqual.tsv is from another script:GraphAlignerBandageScript.ipynb

In [ ]:
df_GraphAligner_Paths

In [ ]:
df_GraphAligner_Paths
df_GraphAligner_Paths.sort_index(inplace=True)
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].astype(str)

In [ ]:
# Function to process the Path values
def process_path(path):
    path = path.replace('<', ' ').replace('>', ' ')
    path = ' '.join(path.split('<'))
    return path

In [ ]:
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].apply(process_path).str.strip()


In [ ]:
df_GraphAligner_Paths
# Create a new column 'Combined_Sequence' that concatenates sequences if Path has two values


In [ ]:
df_GraphAligner_Paths[['Path1', 'Path2']] = df_GraphAligner_Paths['Path Matching'].str.split(' ', expand=True)

In [ ]:
df_GraphAligner_Paths

In [ ]:
# Get the rows where Path 2 is not None
rows_not_none = df_GraphAligner_Paths[df_GraphAligner_Paths['Path2'].notna()]

# Print the resulting dataframe
rows_not_none

In [ ]:
#Get the sequence for Path 1 and Path 2
# Merge df1 and df2 based on matching values in Column1 and Name

merged_df= pd.merge(df_GraphAligner_Paths, df_graph, left_on='Path1', right_on='Name')
merged_df = pd.merge(merged_df, df_graph, left_on='Path2', right_on='Name')

# Concatenate the sequences
merged_df

In [ ]:
extracted_sequences_Path1_df = extract_sequences_Path1(merged_df_Path1)
extracted_sequences_Path1_df
#Save the dataframe to a pdf
extracted_sequences_Path1_df.to_csv("Path1_TestSequences.tsv", sep="\t")